<a href="https://colab.research.google.com/github/VishalPrem1994/AIGenPlayGround/blob/main/IPAdapterBasedSDFaceConsistency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pip install openai numpy opencv-python accelerate transformers ftfy bitsandbytes==0.35.0 gradio natsort safetensors xformers diffusers omegaconf requests
%pip install opencv-python insightface onnxruntime
%pip install git+https://github.com/tencent-ailab/IP-Adapter.git einops

In [ ]:
import insightface
import cv2
from insightface.app import FaceAnalysis
import torch

app = FaceAnalysis(name="buffalo_l", providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
app.prepare(ctx_id=0, det_size=(640, 640))


images = ["/content/drive/MyDrive/stable_diffusion_weights/samples/1.jpg", "/content/drive/MyDrive/stable_diffusion_weights/samples/2.jpg", "/content/drive/MyDrive/stable_diffusion_weights/samples/3.jpg", "/content/drive/MyDrive/stable_diffusion_weights/samples/4.jpg", "/content/drive/MyDrive/stable_diffusion_weights/samples/5.jpg"]

faceid_embeds = []
for image in images:
    image = cv2.imread(image)
    faces = app.get(image)
    faceid_embeds.append(torch.from_numpy(faces[0].normed_embedding).unsqueeze(0).unsqueeze(0))
faceid_embeds = torch.cat(faceid_embeds, dim=1)


In [ ]:
import torch
from diffusers import StableDiffusionPipeline, DDIMScheduler, AutoencoderKL
from PIL import Image

from ip_adapter.ip_adapter_faceid_separate import IPAdapterFaceID

base_model_path = "SG161222/Realistic_Vision_V4.0_noVAE"
vae_model_path = "stabilityai/sd-vae-ft-mse"
ip_ckpt = "/content/drive/MyDrive/stable_diffusion_weights/ip-adapter-faceid-portrait-v11_sd15.bin"
device = "cuda"

noise_scheduler = DDIMScheduler(
    num_train_timesteps=1000,
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear",
    clip_sample=False,
    set_alpha_to_one=False,
    steps_offset=1,
)
vae = AutoencoderKL.from_pretrained(vae_model_path).to(dtype=torch.float16)
pipe = StableDiffusionPipeline.from_pretrained(
    "/content/drive/MyDrive/stable_diffusion_weights/RevAnim",
    torch_dtype=torch.float16,
    scheduler=noise_scheduler,
    vae=vae,
    feature_extractor=None,
    safety_checker=None
)

ip_model = IPAdapterFaceID(pipe, ip_ckpt, device, num_tokens=16, n_cond=5)

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:

# generate image
prompt = "close up portrait  of beautiful woman, strong jawline, (in fashion show, catwalk), saree, hijabi, (transparent silk), beautiful design outfits, smiling, intricate, detailed eyes, taken by nina masic" #@param {type:"string"}
negative_prompt = "(deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck" #@param {type:"string"}
seed  = 34234234 #@param {type:"number"}
width=512 #@param {type:"number"}
height=512 #@param {type:"number"}
images = ip_model.generate(
    prompt=prompt,
    negative_prompt=negative_prompt,
    faceid_embeds=faceid_embeds,
    num_samples=2,
    width=width,
    height=height,
    num_inference_steps=25,
    seed=seed
)


for img in images:
    display(img)